# Bokeh

* Bokeh is an interactive graphing toolkit build on javascript.

* Good tutorial available at http://nbviewer.jupyter.org/github/bokeh/bokeh-notebooks/blob/master/tutorial/00%20-%20Introduction%20and%20Setup.ipynb


* Check out the examples: http://bokeh.pydata.org/en/latest/docs/gallery.html

In [ ]:
import numpy as np
import pandas as pd

# Standard imports 

from bokeh.io import output_notebook, show
from bokeh.plotting import figure


output_notebook()   # load bokeh in the notebook

## Loading some more sample data

**Only needs to be exectude once**

In [ ]:

import bokeh.sampledata
bokeh.sampledata.download()

# Your first Bokeh plot

In [ ]:
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(
    x=np.random.random(100), y=np.random.random(100),
    size=10 * np.random.random(100),  
    line_color='navy',
    fill_color='orange',
    fill_alpha=0.5,  # and alpha
)

show(p) # show the results

* all properties can be vectorized, i.e., hand over a list
* lots of different markers exist, e.g., *asterisk*, *square*, *x*

# Line Plots

In [ ]:
p = figure(plot_width=400, plot_height=400, title="My Line Plot")

p.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=2)

show(p) 

# Plots can be combined

In [ ]:
p = figure(plot_width=400, plot_height=400, title="My Line Plot")

p.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=2)
p.scatter([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=10)

show(p) 

# Categorical Data

In [ ]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts", tools=[])

# Categorical values can also be used as coordinates
p.circle(x=fruits, y=[5, 3, 4, 2, 4, 6])

# Set some properties to make the plot look better
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

# Bar Charts

In [ ]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts", tools=[])

p.vbar(x=fruits, top=[5, 3, 4, 2, 4, 6], width=0.9)

# Set some properties to make the plot look better
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

# Columnar Data

In [ ]:
from bokeh.sampledata.autompg import autompg_clean
autompg_clean.head()

In [ ]:
p = figure()
p.circle(x='weight', y='hp', size='cyl', source=autompg_clean)
show(p)

# Hover

In [ ]:
from bokeh.models import HoverTool

my_hover = HoverTool()

my_hover.tooltips = [
    ('Name of the car', '@name'),
    ('Origin', '@origin'),
]

p = figure()
p.add_tools(my_hover)
p.circle(x='weight', y='hp', color='displ', size='cyl', source=autompg_clean)
show(p)

# Export

## PNG Export

In [ ]:
from bokeh.plotting import figure
from bokeh.sampledata.stocks import AAPL

df = pd.DataFrame(AAPL)
df['date'] = pd.to_datetime(df['date'])

In [ ]:
!conda install selenium  pillow phantomjs-prebuilt

In [ ]:
from bokeh.io import export_png

p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p.line(df['date'], df['close'], color='navy', alpha=0.5)

export_png(p, filename="plot.png")

# Datetime Axes

In [ ]:
from bokeh.sampledata.glucose import data
data.head()

In [ ]:
# reduce data size to one week
week = data.loc['2010-10-01':'2010-10-08']

p = figure(
    x_axis_type="datetime",
    title="Glocose Range",
    plot_height=350,
    plot_width=800,
)
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Value'

p.line(week.index, week.glucose)

show(p)

# Linked Plots

In [ ]:
from bokeh.layouts import gridplot

In [ ]:
from bokeh.models import ColumnDataSource

x = list(range(-20, 21))
y0, y1 = [abs(xx) for xx in x], [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_select,lasso_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=300, height=300)
right.circle('y0', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

In [ ]:
plot_options = dict(width=250, plot_height=250, tools='box_select,lasso_select,pan,wheel_zoom')

# create a new plot
s1 = figure(**plot_options)
s1.circle(x='x', y='y0', size=10, color="navy", source=source)

# create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range, **plot_options)
s2.triangle(x='x', y='y1', size=10, color="firebrick", source=source)

# create a new plot and share only one range
s3 = figure(x_range=s1.x_range, y_range=s1.y_range, **plot_options)
s3.square(x='y0', y='y1', size=10, color="olive", source=source)

p = gridplot([[s1, s2, s3]])

# show the results
show(p)

# Interactivity

In [ ]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = [x*0.005 for x in range(0, 201)]

source = ColumnDataSource(data=dict(x=x, y=x))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)

show(column(slider, plot))

In [ ]:
from bokeh.models import LogColorMapper
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment

palette.reverse()

counties = {
    code: county for code, county in counties.items() if county["state"] == "tx"
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

county_names = [county['name'] for county in counties.values()]
county_rates = [unemployment[county_id] for county_id in counties]
color_mapper = LogColorMapper(palette=palette)

data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
)

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="Texas Unemployment, 2009", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
   )
p.grid.grid_line_color = None

p.patches('x', 'y', source=data,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

show(p)

Great example: https://www.data-dive.com/cologne-bike-rentals-interactive-map-bokeh-dynamic-choropleth

https://automating-gis-processes.github.io/2017/lessons/L5/interactive-map-bokeh.html

https://moderndata.plot.ly/weather-maps-in-python-with-mapbox-gl-xarray-and-netcdf4/